In [ ]:
import requests
from bs4 import BeautifulSoup
import json
from queue import Queue
from urllib.parse import urljoin
# URL of the webpage
url = "https://www.screener.in/explore/"

response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Save raw HTML in a variable
    html = response.text
base_url = "https://www.screener.in"  # Replace with the actual base URL

soup = BeautifulSoup(html, 'html.parser')
sector_links = soup.find_all('a', class_='bordered radius-6 padding-4-12 font-size-14 ink-700')

print(f"Found {len(sector_links)} sector links")

sectors_queue = Queue()

for link in sector_links:
    href = link.get('href')
    absolute_url = urljoin(base_url, href)
    print(f"Processing URL: {absolute_url}")
    
    sectors_queue.put(absolute_url)

print(f"Queue size: {sectors_queue.qsize()}")

queue_list = []
while not sectors_queue.empty():
    queue_list.append(sectors_queue.get())

with open('sectors_queue.json', 'w') as f:
    json.dump(queue_list, f)
    print("Queue saved to sectors_queue.json")

Found 86 sector links
Processing URL: https://www.screener.in/company/compare/00000085/
Processing URL: https://www.screener.in/company/compare/00000001/
Processing URL: https://www.screener.in/company/compare/00000002/
Processing URL: https://www.screener.in/company/compare/00000003/
Processing URL: https://www.screener.in/company/compare/00000004/
Processing URL: https://www.screener.in/company/compare/00000005/
Processing URL: https://www.screener.in/company/compare/00000006/
Processing URL: https://www.screener.in/company/compare/00000066/
Processing URL: https://www.screener.in/company/compare/00000007/
Processing URL: https://www.screener.in/company/compare/00000008/
Processing URL: https://www.screener.in/company/compare/00000009/
Processing URL: https://www.screener.in/company/compare/00000010/
Processing URL: https://www.screener.in/company/compare/00000011/
Processing URL: https://www.screener.in/company/compare/00000012/
Processing URL: https://www.screener.in/company/compar

In [23]:
import requests
from bs4 import BeautifulSoup
import json
from urllib.parse import urljoin
import time
import random
import logging
from queue import Queue


# # Load HTML file for testing purposes. 
# with open('Aerospace & Defence Companies - Screener.htm', 'r', encoding='utf-8') as f:
#     htmltest = f.read()

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def extract_company_links(html):
    # Load HTML file
    soup = BeautifulSoup(html, 'html.parser')

    # Find all company links
    company_links = soup.find_all('a', target="_blank")

    # Create a queue to hold company URLs
    company_queue = Queue()

    # Extract company URLs and add to queue
    for link in company_links:
        company_url = link.get('href')
        company_queue.put(company_url)
        logging.info(f"Added company URL: {company_url}")

    # Print queue size
    logging.info(f"Queue size: {company_queue.qsize()}")

    # Convert queue to list
    company_list = []
    while not company_queue.empty():
        company_list.append(company_queue.get())

    return company_list

In [24]:
def load_visited_sectors():
    try:
        logging.info("Reading visited sectors from visited_sectors.json")
        with open('visited_sectors.json', 'r') as f:
            return set(json.load(f))
    except FileNotFoundError:
        logging.warning("visited_sectors.json not found. Creating new file")
        return set()


def save_visited_sectors(visited_sectors):
    logging.info("Writing visited sectors to visited_sectors.json")
    with open('visited_sectors.json', 'w') as f:
        json.dump(list(visited_sectors), f, indent=4)
    logging.info("Visited sectors written successfully")


def is_sector_visited(sector_name, visited_sectors):
    return sector_name in visited_sectors


def mark_sector_as_visited(sector_name, visited_sectors):
    visited_sectors.add(sector_name)
    save_visited_sectors(visited_sectors)
    
    
def extract_sector_name(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    title_tag = soup.find('title')
    if title_tag:
        title = title_tag.text
        # Remove "- Screener" from the title
        sector_name = title.replace("- Screener", "").strip()
        return sector_name
    else:
        return None

In [ ]:

# Load sector URLs from JSON file
with open('sectors_queue.json', 'r') as f:
    sector_urls = json.load(f)


def test(sector_urls):
    for sector_url in sector_urls:
        sector_name = extract_sector_name()  # Extract sector name from URL
        
        # Check if sector has been visited
        visited_sectors = load_visited_sectors()
        if is_sector_visited(sector_name, visited_sectors):
            logging.info(f"Skipping already visited sector: {sector_name}")
            continue

test(sector_urls)


2024-11-06 16:49:28,188 - INFO - Reading visited sectors from visited_sectors.json
2024-11-06 16:49:28,191 - INFO - Reading visited sectors from visited_sectors.json
2024-11-06 16:49:28,193 - INFO - Reading visited sectors from visited_sectors.json
2024-11-06 16:49:28,195 - INFO - Reading visited sectors from visited_sectors.json
2024-11-06 16:49:28,197 - INFO - Reading visited sectors from visited_sectors.json
2024-11-06 16:49:28,199 - INFO - Reading visited sectors from visited_sectors.json
2024-11-06 16:49:28,202 - INFO - Reading visited sectors from visited_sectors.json
2024-11-06 16:49:28,204 - INFO - Reading visited sectors from visited_sectors.json
2024-11-06 16:49:28,207 - INFO - Reading visited sectors from visited_sectors.json
2024-11-06 16:49:28,209 - INFO - Reading visited sectors from visited_sectors.json
2024-11-06 16:49:28,212 - INFO - Reading visited sectors from visited_sectors.json
2024-11-06 16:49:28,214 - INFO - Reading visited sectors from visited_sectors.json
2024

In [ ]:
def get_queue_from_sector(sector_urls):
    base_url = "https://www.screener.in"  # Replace with the actual base URL

    # User-agent rotation list
    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
        'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:53.0) Gecko/20100101 Firefox/53.0',
        'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
        'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:46.0) Gecko/20100101 Firefox/46.0'
    ]

    last_request_time = time.time()
    max_retries = 3
    time_interval = 15  # seconds

    company_data = {}

    # Iterate over sector URLs
    for sector_url in sector_urls:
        sector_name = sector_url.split('/')[-1]  # Extract sector name from URL
        
        # Check if sector has been visited
        visited_sectors = load_visited_sectors()
        if is_sector_visited(sector_name, visited_sectors):
            logging.info(f"Skipping already visited sector: {sector_name}")
            continue

        company_data[sector_name] = []

        # Add limit=50 parameter to URL
        sector_url_with_limit = f"{sector_url}?limit=50"
        
        logging.info(f"Visiting URL: {sector_url_with_limit}")
        
        retries = 0
        while retries < max_retries:
            try:
                # Calculate delay to ensure time_interval gap between requests
                current_time = time.time()
                time_elapsed = current_time - last_request_time
                delay = max(0, time_interval - time_elapsed)
                logging.info(f"Waiting {delay} seconds before requesting...")
                time.sleep(delay)
                
                # Update last request time
                last_request_time = time.time()
                
                # Randomly select user-agent
                headers = {'User-Agent': random.choice(user_agents)}
                
                # Send HTTP request to sector URL
                response = requests.get(sector_url_with_limit, headers=headers, timeout=10)
                
                if response.status_code == 200:
                    logging.info(f"Successful request to {sector_url_with_limit}")
                    
                    # Extract sector name from HTML content
                    sector_name = extract_sector_name(response.content)
                    if sector_name:
                        # Initialize company_data dictionary with correct sector_name
                        if sector_name not in company_data:
                            company_data[sector_name] = []
                        
                        # Extract company links
                        companies = extract_company_links(response.content)
                        company_data[sector_name].extend(companies)
                        
                        # Save company URLs to JSON file
                        logging.info(f"Saving company data to company_queue.json")
                        with open('company_queue.json', 'w') as f:
                            json.dump(company_data, f, indent=4)
                        
                        # Mark sector as visited
                        mark_sector_as_visited(sector_name, visited_sectors)
                        
                        break  # Exit loop on success
                    else:
                        logging.error("Failed to extract sector name")
                elif response.status_code == 429:
                    logging.warning(f"Rate limit exceeded for {sector_url}. Waiting 60 seconds.")
                    time.sleep(60)  # Wait 60 seconds before retrying
                    retries += 1
                else:
                    logging.error(f"Error {response.status_code} for {sector_url}")
                    retries += 1
            except requests.exceptions.RequestException as e:
                logging.error(f"Request error: {e}")
                retries += 1
                time.sleep(1)  # Wait 1 second before retrying

In [27]:
# Load sector URLs from JSON file
with open('sectors_queue.json', 'r') as f:
    sector_urls = json.load(f)

get_queue_from_sector(sector_urls)

2024-11-06 16:32:55,518 - INFO - Reading visited sectors from visited_sectors.json
2024-11-06 16:32:55,521 - INFO - Visiting URL: https://www.screener.in/company/compare/00000085/?limit=50
2024-11-06 16:32:55,524 - INFO - Waiting 14.99400782585144 seconds before requesting...
2024-11-06 16:33:11,512 - INFO - Successful request to https://www.screener.in/company/compare/00000085/?limit=50
2024-11-06 16:33:11,615 - INFO - Added company URL: /company/HAL/
2024-11-06 16:33:11,617 - INFO - Added company URL: /company/BEL/consolidated/
2024-11-06 16:33:11,617 - INFO - Added company URL: /company/SOLARINDS/consolidated/
2024-11-06 16:33:11,618 - INFO - Added company URL: /company/COCHINSHIP/
2024-11-06 16:33:11,619 - INFO - Added company URL: /company/BDL/
2024-11-06 16:33:11,620 - INFO - Added company URL: /company/GRSE/
2024-11-06 16:33:11,621 - INFO - Added company URL: /company/BEML/consolidated/
2024-11-06 16:33:11,622 - INFO - Added company URL: /company/ZENTEC/consolidated/
2024-11-06 

KeyboardInterrupt: 